In [1]:
import numpy as np
from sklearn import svm
import dv
import cv2
import time
import glob

In [2]:
def loadLabel(path):
    labels = np.genfromtxt(path, delimiter=',', skip_header=1, dtype=int)

    start_time = labels[0][1]
    for i in labels:
        i[1] = i[1]-start_time
        i[2] = i[2]-start_time
        
    return labels

In [3]:
# Return the start time of the given class index.
def start(index, labels):
    return labels[index][1]
 
# Return the end time of the given class index.
def end(index, labels):
    return labels[index][2]

def classID(index, labels):
    return labels[index][0]-1

# Return the class of the current frame. If no gesture then return -1.
def setGesture(currentTime, labels):
    for i in range(len(labels)-1):
        if (currentTime >= start(i, labels) and currentTime <= end(i, labels)):
            return classID(i, labels)
    return -1

In [4]:
def process(X, Y, path, labels):
    offset = 0
    
    with dv.AedatFile(path) as f: 
        for frame in f['frames']:    
            # Set the offset on the first frame.
            if (offset == 0):
                offset = frame.timestamp

            gesture = setGesture((frame.timestamp - offset), labels)
            image = frame.image

            # font
            font = cv2.FONT_HERSHEY_SIMPLEX

            # org
            org = (5, 30)

            # fontScale
            fontScale = 1

            # Blue color in BGR
            color = (255, 0, 0)

            # Line thickness of 2 px
            thickness = 2
            image = cv2.putText(image, str(gesture), org, font, 
                       fontScale, color, thickness, cv2.LINE_AA)

            cv2.imshow('out',image)
            cv2.waitKey(1)

            if (gesture == -1):
                continue
            else:
                Y.append(gesture)

                matrix = frame.image.flatten()
                X.append(matrix)

In [5]:
X = []
Y = []

csv = glob.glob("data/*.csv")
aedat4 = glob.glob("data/*.aedat4")

print(csv)
print(aedat4)

for i in range(len(csv)):
    labels = loadLabel(csv[i])
    process(X, Y, aedat4[i], labels)

['data/user02_fluorescent_led_labels.csv', 'data/user03_fluorescent_led_labels.csv']
['data/user02.aedat4', 'data/user03.aedat4']


In [6]:
print(len(X))
print(len(Y))

4004
4004


In [7]:
clf = svm.SVC()
clf.fit(X, Y)

SVC()

In [8]:
clf.predict([X[1780]])

array([9])